<a href="https://colab.research.google.com/github/enf3tri/machine-learning/blob/main/Resnet%2BConvLSTM(_62).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install keras-metrics
!pip install resnet

In [ ]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.wrappers import TimeDistributed
from tensorflow.keras.optimizers import Adam
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import keras_metrics as km
import pandas as pd 
 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.python.keras.applications.resnet import ResNet50
from sklearn.metrics import classification_report
from keras.applications.resnet import ResNet50

import sys
sys.path.insert(0,'/content/drive/MyDrive/fight-detection-surv-dataset-master/')
classes = ["noFight","fight"]

In [ ]:
seq_len = 20
img_height , img_width = 128, 128

In [ ]:
fight_dataset = pd.read_csv('/content/drive/MyDrive/fight-detection-surv-dataset-master/fight-set.csv')
no_fight_dataset = pd.read_csv('/content/drive/MyDrive/fight-detection-surv-dataset-master/no-fight-set.csv') 
all_dataset = pd.read_csv('/content/drive/MyDrive/fight-detection-surv-dataset-master/all-set.csv')
label_dataset = pd.read_csv('/content/drive/MyDrive/fight-detection-surv-dataset-master/label-set.csv')

data_path = "/content/drive/MyDrive/fight-detection-surv-dataset-master/data2"


train_set = pd.read_csv('/content/drive/MyDrive/fight-detection-surv-dataset-master/train-set.csv')
test_set = pd.read_csv('/content/drive/MyDrive/fight-detection-surv-dataset-master/test-set.csv')


In [ ]:
def frames_extraction(video_path):
    frames_list = []
     
    vidObj = cv2.VideoCapture(video_path)
    count = 1
    frame_count = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
    while count <= seq_len: 
        success, image = vidObj.read() 
        if success:
            image = cv2.resize(image, (img_height, img_width))
            frames_list.append(image)
            count += 1
        else:
            break
 
            
    return frames_list,frame_count

In [ ]:
def create_data(input_dir):
    X = []
    Y = []
    N = 5 
    classes_list = os.listdir(input_dir)
     
    for c in classes_list:
        print(c)
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in files_list:
           frames,frame_count = frames_extraction(os.path.join(os.path.join(input_dir, c), f))
           print(f)               
           if len(frames) == seq_len :
               '''
               for i in range(0,int(frame_count//4)):
                   try:            
                       X.append(frames[i*2])    
                   except:
                       pass
               '''
               A = np.linspace(0, frame_count, num=10)
               
               X.append(frames)          
               y = [0]*len(classes)
               y[classes.index(c)] = 1
               Y.append(y)
    A = np.linspace(0, len(X), num=10) 
    try:
        X = X[A]
    except:
        print("Passing")    
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y



In [ ]:
X, Y = create_data(data_path)
X = X.astype('float16')
Y = Y.astype('float16')
print(len(X))

noFight
nofi087.mp4
nofi097.mp4
nofi106.mp4
nofi060.mp4
nofi069.mp4
nofi107.mp4
nofi061.mp4
nofi016.mp4
nofi141.mp4
nofi056.mp4
nofi010.mp4
nofi111.mp4
nofi006.mp4
nofi041.mp4
nofi015.mp4
nofi090.mp4
nofi149.mp4
nofi066.mp4
nofi150.mp4
nofi011.mp4
nofi124.mp4
nofi048.mp4
nofi083.mp4
nofi119.mp4
nofi071.mp4
nofi095.mp4
nofi042.mp4
nofi114.mp4
nofi012.mp4
nofi133.mp4
nofi024.mp4
nofi074.mp4
nofi059.mp4
nofi121.mp4
nofi130.mp4
nofi054.mp4
nofi100.mp4
nofi044.mp4
nofi078.mp4
nofi068.mp4
nofi029.mp4
nofi047.mp4
nofi127.mp4
nofi072.mp4
nofi115.mp4
nofi102.mp4
nofi076.mp4
nofi138.mp4
nofi088.mp4
nofi040.mp4
nofi018.mp4
nofi094.mp4
nofi101.mp4
nofi025.mp4
nofi023.mp4
nofi055.mp4
nofi132.mp4
nofi038.mp4
nofi019.mp4
nofi001.mp4
nofi082.mp4
nofi122.mp4
nofi085.mp4
nofi080.mp4
nofi064.mp4
nofi053.mp4
nofi070.mp4
nofi073.mp4
nofi126.mp4
nofi140.mp4
nofi063.mp4
nofi137.mp4
nofi092.mp4
nofi058.mp4
nofi105.mp4
nofi039.mp4
nofi099.mp4
nofi028.mp4
nofi065.mp4
nofi146.mp4
nofi008.mp4
nofi116.mp4
nofi075.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, shuffle=True,  random_state=0)

In [ ]:
model = Sequential()
model.add(Conv3D(filters = 4,kernel_size = (1, 5, 5), padding = "same",input_shape = (seq_len, img_height,img_width,3)))
model.add(MaxPooling3D((1,2,2)))
model.add(BatchNormalization())

model.add(Conv3D(filters = 8,kernel_size = (1, 3, 3), padding = "same"))
model.add(MaxPooling3D((1,2,2)))
model.add(BatchNormalization())

model.add(Conv3D(filters = 16,kernel_size = (1, 3, 3), padding = "same"))
model.add(MaxPooling3D((2,2,2)))
model.add(BatchNormalization())

model.add(Conv3D(filters = 32,kernel_size = (1, 3, 3), padding = "same"))
model.add(MaxPooling3D((2,2,2)))
model.add(BatchNormalization())

model.add(Dense(32,activation= "relu"))
model.add(Dropout(0.2))

#model.add(LSTM(344,return_sequences=True,activation = "relu"))
#model.add(LSTM(64,activation = "relu"))
model.add(Flatten())

model.add(Dense(2, activation = "softmax"))

adam = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy',optimizer = adam, metrics=["accuracy"])
 
earlystop = EarlyStopping(monitor="val_loss",patience=5)
callbacks = [earlystop]
 
history = model.fit(x = X_train, y = y_train, epochs=200, batch_size = 4 , shuffle=True, validation_split=0.2, callbacks = callbacks)

Epoch 1/200
51/51 [==============================] - 7s 91ms/step - loss: 0.9112 - accuracy: 0.4265 - val_loss: 0.9444 - val_accuracy: 0.4314
Epoch 2/200
51/51 [==============================] - 4s 76ms/step - loss: 0.7681 - accuracy: 0.5441 - val_loss: 0.8282 - val_accuracy: 0.5098
Epoch 3/200
51/51 [==============================] - 4s 77ms/step - loss: 0.7264 - accuracy: 0.5441 - val_loss: 0.7982 - val_accuracy: 0.5294
Epoch 4/200
51/51 [==============================] - 4s 77ms/step - loss: 0.7383 - accuracy: 0.5441 - val_loss: 0.7283 - val_accuracy: 0.5686
Epoch 5/200
51/51 [==============================] - 4s 77ms/step - loss: 0.6297 - accuracy: 0.6716 - val_loss: 0.7423 - val_accuracy: 0.5490
Epoch 6/200
51/51 [==============================] - 4s 77ms/step - loss: 0.6482 - accuracy: 0.5931 - val_loss: 0.7459 - val_accuracy: 0.5294
Epoch 7/200
51/51 [==============================] - 4s 76ms/step - loss: 0.6521 - accuracy: 0.6078 - val_loss: 0.7271 - val_accuracy: 0.4902
Epoch 

In [ ]:
y_pred = model.predict(X_test)

y_pred = np.argmax(y_pred, axis = 1)
y_test = np.argmax(y_test, axis = 1)
 
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.47      0.38      0.42        21
           1       0.54      0.62      0.58        24

    accuracy                           0.51        45
   macro avg       0.50      0.50      0.50        45
weighted avg       0.51      0.51      0.50        45

